Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

I'm starting with Logistic Regression here.

In [5]:
batch_size = 128
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    bias = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset ,weights) + bias
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels= tf_train_labels)) + beta * tf.nn.l2_loss(weights)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + bias)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + bias)

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.328688
Minibatch accuracy: 8.6%
Validation accuracy: 8.7%
Minibatch loss at step 500: 2.480271
Minibatch accuracy: 78.9%
Validation accuracy: 76.5%
Minibatch loss at step 1000: 1.714416
Minibatch accuracy: 80.5%
Validation accuracy: 79.0%
Minibatch loss at step 1500: 0.980087
Minibatch accuracy: 82.8%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 0.833194
Minibatch accuracy: 86.7%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 0.821500
Minibatch accuracy: 82.0%
Validation accuracy: 81.5%
Minibatch loss at step 3000: 0.754751
Minibatch accuracy: 82.0%
Validation accuracy: 81.8%
Test accuracy: 88.9%


Now I will try with a 1-layer NN

In [8]:
batch_size = 128
num_hidden_nodes = 1024

In [9]:
graph = tf.Graph()

with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    bias = tf.Variable(tf.zeros([num_hidden_nodes]))
    
    weights_1 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    bias_1 = tf.Variable(tf.zeros([num_labels]))
    
    
    hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset ,weights) + bias)
    logits = tf.matmul(hidden_1, weights_1) + bias_1
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels= tf_train_labels)) + beta * tf.nn.l2_loss(weights)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + bias)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden_1, weights_1) + bias_1)
    test_hidden_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + bias)
    test_prediction = tf.nn.softmax(tf.matmul(test_hidden_1, weights_1) + bias_1)
    

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 724.625244
Minibatch accuracy: 6.2%
Validation accuracy: 30.2%
Minibatch loss at step 500: 198.096680
Minibatch accuracy: 80.5%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 115.309296
Minibatch accuracy: 82.0%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 69.088181
Minibatch accuracy: 89.8%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 41.224514
Minibatch accuracy: 89.8%
Validation accuracy: 84.2%
Minibatch loss at step 2500: 25.115265
Minibatch accuracy: 88.3%
Validation accuracy: 85.1%
Minibatch loss at step 3000: 15.398703
Minibatch accuracy: 86.7%
Validation accuracy: 86.0%
Test accuracy: 93.0%


Here as it can be seen the regularisation term has helped in attaining a better test accuracy and has been able to genralise better.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Here I will try to restrict data to a few batches and aim to overfit the data, using the same ComputeGraph as above.

In [11]:
num_steps = 40
num_batches = 3
batch_size = 128

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = ((step % num_batches) * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 737.794067
Minibatch accuracy: 7.0%
Validation accuracy: 38.9%
Minibatch loss at step 1: 1788.830200
Minibatch accuracy: 39.1%
Validation accuracy: 28.9%
Minibatch loss at step 2: 1981.261108
Minibatch accuracy: 28.9%
Validation accuracy: 34.9%
Minibatch loss at step 3: 1187.243286
Minibatch accuracy: 32.0%
Validation accuracy: 41.8%
Minibatch loss at step 4: 957.557007
Minibatch accuracy: 38.3%
Validation accuracy: 46.9%
Minibatch loss at step 5: 540.119202
Minibatch accuracy: 64.8%
Validation accuracy: 64.1%
Minibatch loss at step 6: 511.764374
Minibatch accuracy: 67.2%
Validation accuracy: 65.7%
Minibatch loss at step 7: 415.002258
Minibatch accuracy: 71.1%
Validation accuracy: 67.2%
Minibatch loss at step 8: 360.369659
Minibatch accuracy: 89.1%
Validation accuracy: 71.0%
Minibatch loss at step 9: 389.745789
Minibatch accuracy: 82.8%
Validation accuracy: 68.6%
Minibatch loss at step 10: 341.574493
Minibatch accuracy: 84.4%
Validation accuracy: 6

As can be seen the validation set is stuck at 74.4% because of overfitting the data while batch accuracy has increased to 100%.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

Here I will try using a dropout on the same overfitting graph and see how it works.

In [12]:
graph = tf.Graph()

with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    bias = tf.Variable(tf.zeros([num_hidden_nodes]))
    
    weights_1 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    bias_1 = tf.Variable(tf.zeros([num_labels]))
    
    
    hidden_1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset ,weights) + bias), 0.5)
    logits = tf.matmul(hidden_1, weights_1) + bias_1
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels= tf_train_labels)) + beta * tf.nn.l2_loss(weights)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + bias)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden_1, weights_1) + bias_1)
    test_hidden_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + bias)
    test_prediction = tf.nn.softmax(tf.matmul(test_hidden_1, weights_1) + bias_1)
    

In [26]:
num_steps = 40
num_batches = 3
batch_size = 128

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = ((step % num_batches) * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 783.006714
Minibatch accuracy: 7.8%
Validation accuracy: 30.7%
Minibatch loss at step 1: 1231.511230
Minibatch accuracy: 24.2%
Validation accuracy: 38.0%
Minibatch loss at step 2: 1501.755371
Minibatch accuracy: 43.8%
Validation accuracy: 32.2%
Minibatch loss at step 3: 1060.729004
Minibatch accuracy: 32.8%
Validation accuracy: 42.5%
Minibatch loss at step 4: 851.424683
Minibatch accuracy: 48.4%
Validation accuracy: 57.2%
Minibatch loss at step 5: 483.585022
Minibatch accuracy: 68.0%
Validation accuracy: 69.0%
Minibatch loss at step 6: 534.817627
Minibatch accuracy: 68.8%
Validation accuracy: 65.3%
Minibatch loss at step 7: 464.048340
Minibatch accuracy: 73.4%
Validation accuracy: 70.6%
Minibatch loss at step 8: 354.078125
Minibatch accuracy: 81.2%
Validation accuracy: 72.4%
Minibatch loss at step 9: 399.140991
Minibatch accuracy: 78.1%
Validation accuracy: 72.4%
Minibatch lo

As can be seen above, the test accuracy is 83.2% and is an improvement on the previous solution. This shows that dropout does indeed decrease overfitting phenomenon.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Here I will try to build a 3 layered neural network to complete the task and will also use relu activations and dropout of .5 between every hidden layer

In [34]:
graph = tf.Graph()
num_hidden_nodes = 1024
num_hidden_nodes_1 = 512
num_hidden_nodes_2 = 256

with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes], stddev= np.sqrt(2.0 / (image_size * image_size))))
    bias = tf.Variable(tf.zeros([num_hidden_nodes]))
    
    weights_1 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_hidden_nodes_1], stddev= np.sqrt(2.0 / num_hidden_nodes)))
    bias_1 = tf.Variable(tf.zeros([num_hidden_nodes_1]))
    
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes_1, num_hidden_nodes_2], stddev= np.sqrt(2.0 / num_hidden_nodes_1)))
    bias_2 = tf.Variable(tf.zeros([num_hidden_nodes_2]))
    
    weights_3 = tf.Variable(tf.truncated_normal([num_hidden_nodes_2, num_labels], stddev= np.sqrt(2.0 / num_hidden_nodes_2)))
    bias_3 = tf.Variable(tf.zeros([num_labels]))
    
    
    hidden_1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset ,weights) + bias), 0.5)
    hidden_2 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden_1 ,weights_1) + bias_1), 0.5)
    hidden_3 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden_2 ,weights_2) + bias_2), 0.5)
    logits = tf.matmul(hidden_3, weights_3) + bias_3
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels= tf_train_labels))
    
    learning_rate = tf.train.exponential_decay(0.5, global_step, 5000, 0.80, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    v_hidden_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + bias)
    v_hidden_2 = tf.nn.relu(tf.matmul(v_hidden_1, weights_1) + bias_1)
    v_hidden_3 = tf.nn.relu(tf.matmul(v_hidden_2, weights_2) + bias_2)
    v_logits = tf.matmul(v_hidden_3, weights_3) + bias_3
    validation_prediction = tf.nn.softmax(v_logits)
    
    t_hidden_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + bias)
    t_hidden_2 = tf.nn.relu(tf.matmul(t_hidden_1, weights_1) + bias_1)
    t_hidden_3 = tf.nn.relu(tf.matmul(t_hidden_2, weights_2) + bias_2)
    t_logits = tf.matmul(t_hidden_3, weights_3) + bias_3
    test_prediction = tf.nn.softmax(t_logits)
    

In [35]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(validation_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 2.865965
Minibatch accuracy: 10.2%
Validation accuracy: 26.2%
Minibatch loss at step 500: 0.483439
Minibatch accuracy: 88.3%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.622036
Minibatch accuracy: 84.4%
Validation accuracy: 84.5%
Minibatch loss at step 1500: 0.473837
Minibatch accuracy: 85.2%
Validation accuracy: 85.6%
Minibatch loss at step 2000: 0.402121
Minibatch accuracy: 88.3%
Validation accuracy: 86.3%
Minibatch loss at step 2500: 0.497054
Minibatch accuracy: 82.8%
Validation accuracy: 86.2%
Minibatch loss at step 3000: 0.496634
Minibatch accuracy: 84.4%
Validation accuracy: 86.6%
Minibatch loss at step 3500: 0.550857
Minibatch accuracy: 80.5%
Validation accuracy: 87.3%
Minibatch loss at step 4000: 0.431133
Minibatch accuracy: 89.8%
Validation accuracy: 87.2%
Minibatch loss at step 4500: 0.503740
Minibatch accuracy: 85.9%
Validation accuracy: 86.8%
Minibatc

This is the best accuracy I've gotten after a lot of tuning with the hyper parameters